In [28]:
# importing libraries
import pandas as pd 
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import joblib

In [29]:
list=['Biopsy','Smokes',
'HC',
'IUD',
'STDs',
'Dx:CIN',
'Dx',
'Hinselmann',
'Schiller',
'Citology',
'Age',
'NSP',
'FSI',
'NPRE',
'SY',
'SmokesPY',
]

In [30]:
train= pd.read_csv("train_datasetset.csv")
train=train[list]
train.head()

,Biopsy,Smokes,HC,IUD,STDs,Dx:CIN,Dx,Hinselmann,Schiller,Citology,Age,NSP,FSI,NPRE,SY,SmokesPY
0,0,0,1,0,0,0,0,0,0,0,0.042254,0.037037,0.181818,0.181818,0.0,0.0
1,1,0,0,0,0,0,0,1,1,1,0.272095,0.074074,0.359766,0.181818,0.0,0.0
2,1,0,0,1,0,0,0,0,1,0,0.501376,0.100043,0.293102,0.377220,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,0.070423,0.000000,0.227273,0.090909,0.0,0.0
4,1,0,1,0,1,0,0,0,1,0,0.188762,0.022241,0.481724,0.145502,0.0,0.0


In [31]:
train=train.fillna(0)
X_train=train.drop(['Biopsy'],axis = 1 )
y_train=train['Biopsy']

In [32]:
test= pd.read_csv("unseen_dataset.csv")
test=test[list]
y_test=test['Biopsy']
X_test=test.drop(['Biopsy'],axis = 1 )

In [33]:

model_params = {
    
    'RF': {
        'model': RandomForestClassifier(),
        'params' : {
            'max_depth' : [10,20,30],

        }
    },
    'LR' : {
        'model': LogisticRegression(),
        'params': {
            "solver" : ['newton-cg','lbfgs','liblinear','sag','saga']

                     
        }}
    ,
    'DT': {
        'model': DecisionTreeClassifier(),
        'params': {
       
            'max_depth' : [1,2,3,4,5,6,7]
        }
        
    }  
    ,
   
 'SVM': {
     'model': SVC(probability=True),
    'params':  {
                'kernel': ['linear','rbf','poly',]
    }},

    
  'NB': {
     'model': GaussianNB(),
    'params':  {
      'var_smoothing': np.logspace(0,-9, num=100)
    }
 },
    
    'AdaBoost': {
     'model': AdaBoostClassifier(),
      "params" :{
   
    'n_estimators': [10, 50]}
   } 
  
      
    
   } 

   

In [34]:
for model_name, mp in model_params.items():
    print(model_name)

RF
LR
DT
SVM
NB
AdaBoost


In [35]:
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average='weighted'),
           'recall' : make_scorer(recall_score, average='weighted'), 
           'f1' : make_scorer(f1_score, average='weighted')}


In [ ]:
import warnings
warnings.filterwarnings('ignore')
scores = []
model={}
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=10, scoring =make_scorer(f1_score, average='weighted'),return_train_score=False)
    clf.fit(X_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
   
    model[model_name]=clf.best_estimator_
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

In [ ]:
clf = [('RF', model['RF']),('LR', model['LR']), ('DT', model['DT']), ('SVM', model['SVM'])] 
stack_model = StackingClassifier( estimators = clf,final_estimator =model['LR'])
stacking = stack_model.fit(X_train,y_train)

In [ ]:
def evaluate_models(models, X_train, X_val, y_train, y_val):
 
    scores = list()
    for name, model in models:

        yhat = model.predict(X_val)
        acc = accuracy_score(y_val, yhat)
        
        scores.append(acc)
        # report model performance
    return scores
 

In [ ]:
Voting = [('LR', model['LR']), ('DT', model['DT']), ('SVM', model['SVM']), 
  ('NB', model['NB'])] 



In [ ]:
from sklearn.ensemble import VotingClassifier
Voting_soft = VotingClassifier(estimators=clf, voting='soft')
Voting_soft.fit(X_train, y_train)

In [ ]:
from sklearn.ensemble import BaggingClassifier
Bagging = BaggingClassifier(DecisionTreeClassifier(), n_estimators=2)
Bagging1 = Bagging.fit(X_train, y_train)

In [ ]:
model.update({'Bagging':Bagging1})
model.update({'Voting':Voting_soft} )
model.update({'Stacking':stacking} )


In [ ]:
finaltrain = pd.DataFrame([])
finalSTD = pd.DataFrame([])
ReultofTest=pd.DataFrame([])


for model_name, i in model.items():   
    Model=i
    AccuracyTrain=[]
    PrecisionTrain=[]
    RecallTrain=[]
    F1Train=[]
    Roc_auc_Train=[]
    AccuracyTest=[]
    PrecisionTest=[]
    RecallTest=[]
    F1Test=[]
    Roc_auc_Test=[]
    AccuracyTrain=[]
    
    SDAccuracyTrain=[]
    SDPrecisionTrain=[]
    SDReacllTrain=[]
    SDF1Train=[]
    
    
    AccuracyTest=[]
    PrecisionTest=[]
    RecallTest=[]
    F1Test=[]
    
    for i in range(0,1):
        if i==0:
            r=70
        elif i== 1:
            r=80
        elif i== 2:
            r=90

        SFold = StratifiedKFold(n_splits=10,  shuffle=True,random_state=r)
        Train_Score = cross_validate(Model, X_train, y_train, scoring=scoring,  cv=SFold)
        AccuracyTrain.append(round(100*np.mean(Train_Score['test_accuracy']), 2))
        PrecisionTrain.append(round(100*np.mean(Train_Score['test_precision']), 2))
        RecallTrain.append(round(100*np.mean(Train_Score['test_recall']), 2))
        F1Train.append(round(100*np.mean(Train_Score['test_f1']), 2))
       

        
       
        SDAccuracyTrain.append(round(100*np.std(Train_Score['test_accuracy']), 2))
        
        SDPrecisionTrain.append(round(100*np.std(Train_Score['test_precision']), 2))
        
        SDReacllTrain.append(round(100*np.std(Train_Score['test_recall']), 2))
       
        SDF1Train.append(round(100*np.std(Train_Score['test_f1']), 2))
        
        
        y_pred = Model.predict(X_test)
             
        Accurcy_Test= accuracy_score(y_test,y_pred)
        Precision_Test=precision_score(y_test, y_pred, average='weighted')
        Recall_Test=recall_score(y_test, y_pred, average='weighted')
        F1_Test=f1_score(y_test, y_pred, average='weighted') 
        
        
        AT= str(round(np.mean(AccuracyTrain),2))
        SDATrain=str(round(np.mean(SDAccuracyTrain),2))
        Accuracy_SD=AT +"±" +SDATrain
        Accuracy_SD
        PT= str(round(np.mean(PrecisionTrain),2))
        SDPTrain=str(round(np.mean(SDPrecisionTrain),2))
        PrecisionTrain_SD=PT +"±" +SDPTrain
        
        
        RT= str(round(np.mean(RecallTrain),2))
        SDReacllTrain1=str(round(np.mean(SDReacllTrain),2))
        RecallTrain_SD=RT +"±" +SDReacllTrain1
       
        
        FT= str(round(np.mean(F1Train),2))
        SDFTrain=str(round(np.mean( SDF1Train),2))
        FTrain_SD=FT +"±" +SDFTrain
        
        AccuracyTest.append(round(100*Accurcy_Test, 2))
        PrecisionTest.append(round(100*Precision_Test, 2))
        RecallTest.append(round(100*Recall_Test, 2))
        F1Test.append(round(100*F1_Test, 2)) 
       
      
        finaltrain = finaltrain._append({ 'model':model_name,
                                        'AccuracyTrain' : round(np.mean(AccuracyTrain),2),
                                        'PrecisionTrain':round(np.mean(PrecisionTrain),2),
                                        'RecallTrain':round(np.mean(RecallTrain),2),  
                                        'F1Train':round(np.mean(F1Train),2),
                                      
                                       } , ignore_index=True)
        finaltrain.reindex(['model','AccuracyTrain','PrecisionTrain','RecallTrain','F1Train'], axis=1)
        
       
     
 
        finalSTD = finalSTD._append({ 'model':model_name,
                                        'AccuracyTrain' : Accuracy_SD,
                                        'PrecisionTrain':PrecisionTrain_SD,
                                        'RecallTrain':RecallTrain_SD,  
                                        'F1Train':FTrain_SD,
                                       
                                       } , ignore_index=True)
        
        finalSTD.reindex(['model','AccuracyTrain','PrecisionTrain','RecallTrain','F1Train'], axis=1)
    
        ReultofTest=ReultofTest._append({'model':model_name,
                                        'AccuracyTest' : round(np.mean(AccuracyTest),2),
                                        'PrecisionTest':round(np.mean(PrecisionTest),2),
                                        'RecallTest' : round(np.mean(RecallTest),2),'F1Test':round(np.mean(F1Test),2),
                                  
                                       }, ignore_index=True)
        ReultofTest.reindex(['model','AccuracyTest','PrecisionTest','RecallTest','F1Test'],axis=1)
       

# Result of training

In [ ]:
finalSTD

# Result of testing

In [ ]:
h=ReultofTest.iloc[:, 1::]
from IPython.display import HTML

HTML(h.to_html(index=False))

In [ ]:
classifiers = {'RF': model['XGB'],'LR': model['LR'], 'DT': model['DT'], 
               'SVM': model['SVM'],   'NB': model['NB'], 
               
               'Voting': model['Voting'],
               'Bagging': model['Bagging'],
               'AdaBoost': model['AdaBoost'],
               'Stacking': model['Stacking']}

# Define a result table as a DataFrame
result_table = pd.DataFrame(columns=['classifiers', 'fpr','tpr','auc'])

# Train the models and record the results
for k,v in classifiers.items():
   

    
    predicted = v.predict(X_test)

    #predicted = np.argmax(predicted_proba, axis = 1)
    fpr, tpr, _ = roc_curve(y_test,  predicted)
    auc = round(roc_auc_score(y_test, predicted),5)*100
    
    result_table = result_table._append({'classifiers':k,
                                        'fpr':fpr, 
                                        'tpr':tpr, 
                                        'auc':auc}, ignore_index=True)

# Set name of the classifiers as index labels
result_table.set_index('classifiers', inplace=True)


In [ ]:
fig = plt.figure(figsize=(8,6))

for i in result_table.index:
    plt.plot(result_table.loc[i]['fpr'], 
             result_table.loc[i]['tpr'], 
             label="{}, AUC={:.3f}".format(i, result_table.loc[i]['auc']))
    
plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("Flase Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

plt.show()